In [ ]:
###########################################################
##[F] Find the MASK file with the biggest white pixel val##
###########################################################

import cv2
import os
import pydicom

path = 'D:/TCIA_test'

bestImg={}
bestSize={}
percentPix={}
for pat_dir in os.listdir(path):
    for x in os.listdir(os.path.join(path,pat_dir,'MASK')):
        _,ext=os.path.splitext(x) #split into extension name and file's name
        if ext=='.png' or ext=='.jpg':
            img=cv2.imread(os.path.join(path,pat_dir,'MASK',x),cv2.IMREAD_GRAYSCALE)
        else:
            continue
        numWhitePix = 0 
        for i in range(img.shape[0]):
            for j in range(img.shape[1]):
                if img[i][j]==255: #white pixel
                    numWhitePix+=1
        percentPix.update({x : (numWhitePix / img.size) * 100.0}) #save patient ID and white pixel's percentage
    print(pat_dir,': ',max(percentPix,key=percentPix.get))
    bestImg.update({pat_dir:max(percentPix,key=percentPix.get)})
    bestSize.update({pat_dir:max(percentPix.values())})
    percentPix={}
print(bestImg)
print(bestSize)

In [ ]:
###create a patch file per every patient###
for pat_dir in list(os.listdir(path)):
    #mask filename, ct filename, patch filename
    mask_filename,ext=os.path.splitext(bestImg[pat_dir])
    ct_filename=mask_filename.replace('mask_','000')
    ct_filename=str(int(ct_filename)-1).zfill(6)+'.png'
    patch_filename=mask_filename.replace('mask','patch')+'.png'
    
    #load mask and ct files
    mask=cv2.imread(os.path.join(path,pat_dir,'MASK',bestImg[pat_dir]),cv2.IMREAD_GRAYSCALE)
    ct=cv2.imread(os.path.join(path,pat_dir,'CT',ct_filename),cv2.IMREAD_GRAYSCALE)  
    
    #find center coordinates of nodule 
    M=cv2.moments(mask)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    
    #crop ct image depending on its center coordinates(=create the patch)
    patch=ct[cY-int(patch_size/2):cY+int(patch_size/2),cX-int(patch_size/2):cX+int(patch_size/2)]
    #patch_mask=mask[cY-int(patch_size/2):cY+int(patch_size/2),cX-int(patch_size/2):cX+int(patch_size/2)]
    
    #write the patch file on the patch directory
    cv2.imwrite(os.path.join(path,pat_dir,'patch',patch_filename),patch)
    #plt.imshow(patch,cmap='gray')
    #plt.imshow(patch_mask,cmap='gray')

In [5]:
########################################################
##sort CT image with instance num and save as PNG file##
########################################################

import cv2
import os
import pydicom
import numpy as np

path = 'D:/TCIA_test/'
PNG = True

dict_dcm_IN = {}
tmp_sorted_IN = []

for path_dir in os.listdir(path):
    for x in os.listdir(os.path.join(path, path_dir, 'CT')):
        #loading DCM file's Instance Number
        fileName, fileExt = os.path.splitext(x) #splilt file into file's name and file's esxtension name
        if fileExt == '.dcm':
            dcm_file = pydicom.dcmread(os.path.join(path, path_dir, 'CT', x))
            dcm_IN = int(dcm_file.InstanceNumber) #instance number, convert to int for sorting
            dict_dcm_IN.update({fileName : dcm_IN})
            tmp_sorted_IN = sorted(dict_dcm_IN.items(), key = lambda item:item[1]) #sorted with instance num
        else:
            continue
    for i in range(len(tmp_sorted_IN)):
        dcm_img = pydicom.read_file(os.path.join(path, path_dir, 'CT', tmp_sorted_IN[i][0]+'.dcm'))
        pix_arr = dcm_img.pixel_array.astype(float)
        pix_arr_scaled = (np.maximum(pix_arr, 0) / pix_arr.max()) * 255.0
        pix_arr_scaled = np.uint8(pix_arr_scaled)
        cv2.imwrite(os.path.join(path, path_dir, 'CT', 'CT_' + str(i) + '.png'), pix_arr_scaled) #rename connected with mask file's name
    dict_dcm_IN = {} #reset to reuse
    tmp_sorted_IN = [] #reset to reuse
    
#print(dict_dcm_IN)
#print("----------")
#print(tmp_sorted_IN)

{}
----------
[]
